In [1]:
import time
from threading import Thread
import logging
import os
from datetime import datetime
import re

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

In [3]:
chrome_options = Options()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--disable-session-crashed-bubble")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--suppress-message-center-popups")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("disable-infobars")

chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--window-size=1920x1080")

# chrome_options.add_argument("--no-first-run")
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
# chrome_options.add_experimental_option('useAutomationExtension', False)
# chrome_options.add_experimental_option("excludeSwitches", ["enable-logging", "test-type"])

In [4]:
CWD = os.path.dirname(os.path.realpath("./"))
tdt = datetime.now()
td = datetime.now().strftime('%m-%d-%Y')
FIVEM_URL = 'https://servers.fivem.net/servers'
BASE_URL = 'https://servers.fivem.net/servers/detail/{}'
FIVEM_OUTPUT = os.path.join(CWD, 'FiveMScraper.csv')
# last_index = 0
print("Done")

Done


In [5]:
log_file_path = os.path.join(CWD, f"fivem_scraper.log")
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
file_handler = logging.FileHandler(filename=log_file_path, mode="w", encoding="UTF-8")
file_handler.setFormatter(
    logging.Formatter("%(asctime)s-%(levelname)s: Func-%(funcName)s : Line-%(lineno)d : %(message)s"))
logger.addHandler(file_handler)
logger.addHandler(logging.StreamHandler())
logger.debug(f"\n{'-' * 21} FiveM Scraper == Started at -> {tdt.strftime('%d-%m-%Y %I:%M:%S %p')} {'-' * 21}")
logger.info(f"Current Working Directory: {CWD}")
logger.info(f"Current Date & Time: {tdt.strftime('%d-%m-%Y %I:%M:%S %p')}")
logger.info(f"Log File Path: {log_file_path}")
logger.info(f"OUTPUT File Path: {FIVEM_OUTPUT}")
# ------------- Starter Code -------------
logging.getLogger('WDM').setLevel(logging.CRITICAL)
os.environ['WDM_LOG'] = '0'
# DRIVER_PATH = ChromeDriverManager(version="114.0.5735.90").install()
# logger.info(f"Chrome Driver File Path: {log_file_path}")



--------------------- FiveM Scraper == Started at -> 25-10-2023 05:56:32 PM ---------------------
Current Working Directory: C:\Users\muham\Desktop
Current Date & Time: 25-10-2023 05:56:32 PM
Log File Path: C:\Users\muham\Desktop\fivem_scraper.log
OUTPUT File Path: C:\Users\muham\Desktop\FiveMScraper.csv


In [6]:
# driver = webdriver.Chrome(service=Service(r"chromedriver.exe"), options=chrome_options)

In [7]:
def push_data(data):
    if not os.path.exists(FIVEM_OUTPUT):
        df = pd.DataFrame(
            columns=["Server Name", "FiveM Server Link", "Developer", "Game type", "Map", "OneSync", "Language",
                     "Tags", "DLC level", "sv_purelevel", "Owner", "DiscordLink", "Extracted_Link", "Resources Section"])
        df.loc[df.shape[0]] = data
        df.to_csv(FIVEM_OUTPUT, index=False)  # Save as CSV

    else:
        df = pd.read_csv(FIVEM_OUTPUT)  # Read existing CSV
        df.loc[df.shape[0]] = data
        df.to_csv(FIVEM_OUTPUT, index=False)  # Save as CSV

In [8]:
def fetch_data(server_id):
    url = BASE_URL.format(server_id)
    logger.info(f"{url}")
    data = ["", "", "", "", "", "", "", "", "", "", "", "", "",""]
    
    driver.get(url)

    try:
        data[1] = url
        server_name_element = driver.find_element(By.XPATH, '(//span)[24]')
        server_name = server_name_element.text
        print(server_name)
        
        logger.info(f"Server Name: {server_name}")
        data[0] = server_name

        pattern = r"discord\.gg/[a-zA-Z0-9]+(?:\.[a-zA-Z]+)?"
        links = re.findall(pattern, server_namez)
        if links:
            print("Found link(s):")
            print(links)
            data[12] = links
        else:
            print("no links")
            data[12] = " "
    except:
        pass

    resource_list = []
    resources = WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.XPATH, '(//ul)[3]/li')))
    for resource in resources:
        resource_list.append(resource.text)
        #logger.info(f"Resource: {resource.text}")
    data[13] = resource_list
    logger.info(f"Resources: {', '.join(resource_list)}")        

    tags = WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.XPATH, '(//ul)[2]/li')))

    for tag in tags:
        key = WebDriverWait(tag, 60).until(
            EC.presence_of_element_located((By.XPATH, './span[1]')))

        value = WebDriverWait(tag, 60).until(
            EC.presence_of_element_located((By.XPATH, './span[2]')))

        # ["Server Name", "FiveM Server Link", "Resources Section", "Game type", "Map", "OneSync", "Language",
        #  "Tags", "DLC level", "sv_purelevel", "Owner", "DiscordLink", "Developer"]
        key = key.text
        value = value.text
        logger.info(f"{key}: {value}")
        if key == 'Game type':
            data[3] = value
        elif key == 'OneSync':
            data[4] = value
        elif key == 'Map':
            data[5] = value
        elif key == 'Language':
            data[6] = value
        elif key == 'Tags':
            data[7] = value
        elif key == 'DLC level':
            data[8] = value
        elif key == 'sv_purelevel':
            data[9] = value
        elif key == 'Owner':
            data[10] = value
        elif key == 'Developer':
            data[2] = value
        elif key == 'DiscordLink':
            data[11] = value
        # logger.info(f"Tag: {tag.text}")

    print(data)
    push_data(data)

In [9]:
# fetch_data('xdqklg')

In [10]:
def scrape_single_servers():
    df = pd.read_csv('All Links.csv', sep='\t', names=['server_id'])
    print("""
    Do you want to provide some custom index to start the script from?
    """)
    cindex = input("If yes, type y ")
    if cindex == 'y':
        new_index = int(input("Provide a custom index: "))
        df = df[df.index>new_index]
        last_index = new_index

    for i, item in df.iterrows():
        try:
            server_input = item['server_id']
            print(f"Scraping server {i}, id is {server_input}")
            print(server_input)
            fetch_data(server_input)
            print(f"The last index is: {i}")
        except Exception as Ex:
            print("An error occured scraping single servers.")
            print(Ex)

In [11]:
print("By default, this script scrapes data from start of file to the end.")
# print(f"The last index previously saved for some data is {last_index}.")

By default, this script scrapes data from start of file to the end.


In [12]:
scrape_single_servers()


    Do you want to provide some custom index to start the script from?
    
If yes, type y 
Scraping server 0, id is 23o7lr
23o7lr
The last index is: 0
Scraping server 1, id is 8qvyrb
8qvyrb
The last index is: 1
Scraping server 2, id is gzr7mx
gzr7mx
The last index is: 2
Scraping server 3, id is xdqklg
xdqklg
The last index is: 3
Scraping server 4, id is zgag6d
zgag6d
The last index is: 4
Scraping server 5, id is wya6bj
wya6bj
The last index is: 5
Scraping server 6, id is oop8dy
oop8dy
The last index is: 6
Scraping server 7, id is mo9kod
mo9kod
The last index is: 7
Scraping server 8, id is 8grqp5
8grqp5
The last index is: 8
Scraping server 9, id is kx5la6
kx5la6
The last index is: 9
Scraping server 10, id is 269jld
269jld
The last index is: 10
Scraping server 11, id is 8lpkzm
8lpkzm
The last index is: 11
Scraping server 12, id is zmpy54
zmpy54
The last index is: 12
Scraping server 13, id is m4oll9
m4oll9
The last index is: 13
Scraping server 14, id is 8kl7a3
8kl7a3
The last index is: 